In [4]:
from pathlib import Path

import numpy as np

from pydub import AudioSegment

DATA_PATH = Path('data-kaggle')
DATA_TRAIN_PATH = DATA_PATH / 'train'
DATA_TEST_PATH = DATA_PATH / 'test'
BG_FILES = list(DATA_TRAIN_PATH.glob('audio/_background_noise_/*.wav'))

SAMPLE_RATE = 16000


def generate_silence(n, path):
    """Generated n samples of silence from background noises with random crop and volue.
    Combines up to 2 random samples from background noises."""
    for j in range(n):
        choice = np.random.randint(1, 3)
        
        ## choose background noises to mix
        chosen_bg_files = [BG_FILES[i] for i in np.random.randint(6, size=choice)]
        
        silence = AudioSegment.silent(duration=1000, frame_rate=SAMPLE_RATE)
        
        bg_files = [AudioSegment.from_wav(x) for x in chosen_bg_files]
    
        for bg_file in bg_files:
            random_slice_idx = np.random.randint(0, bg_file.duration_seconds) * 1000
            random_bg_slice = bg_file[random_slice_idx:random_slice_idx + 1000]
            silence = silence.overlay(random_bg_slice + np.random.randint(-32 * choice, -15 * choice))
    
        file_path = str(path / f"audio/silence/{j}_00_00.wav")
        silence.export(path / f"audio/silence/{j}_00_00.wav", format="wav")
        if np.random.random() < 0.3: 
            with open(DATA_TRAIN_PATH / "testing_list.txt", "a") as f:
                f.write(f"silence/{j}_00_00.wav\n")

In [5]:
generate_silence(6000, DATA_TRAIN_PATH)